In [2]:
import tabula
import textract
import re
import pymysql
from os import listdir
from os.path import isfile, join
import pandas as pd
from IPython.display import display
import collections
import warnings
import db_conn
warnings.filterwarnings("ignore")

In [3]:
CONN = db_conn.get_connection()

In [3]:
root_dir ='/Users/gracelee/football_worldcup' #group phase
# root_dir ='/Users/gracelee/football_worldcup_knock'#knockout phase
onlyfiles = [f for f in listdir(root_dir) if isfile(join(root_dir, f))]
onlyfiles = [root_dir+'/'+x for x in onlyfiles if '-start-' in x]
# onlyfiles

In [4]:
onlyfiles.__len__()

48

In [5]:
team_list = []
targetfiles = []
for f in onlyfiles:
    cs = f.split('-start')[0].split('-')[-2:]
    if not team_list.__contains__(cs[0]) and not team_list.__contains__(cs[1]):
        team_list.append(cs[0])
        team_list.append(cs[1])
        targetfiles.append(f)

In [6]:
countries = {'russia':'1',
            'saudi arabia':'2',
            'egypt':'3',
            'uruguay':'4',
            'portugal':'5',
            'spain':'6',
            'morocco':'7',
            'ir iran':'8',
            'france':'9',
            'australia':'10',
            'peru':'11',
            'denmark':'12',
            'argentina':'13',
            'iceland':'14',
            'croatia':'15',
            'nigeria':'16',
            'brazil':'17',
            'switzerland':'18',
            'costa rica':'19',
            'serbia':'20',
            'germany':'21',
            'mexico':'22',
            'sweden':'23',
            'korea republic':'24',
            'belgium':'25',
            'panama':'26',
            'tunisia':'27',
            'england':'28',
            'poland':'29',
            'senegal':'30',
            'colombia':'31',
            'japan':'32'}

In [ ]:
tc = []
ldf = collections.OrderedDict()
odf = []

for z in range(len(targetfiles)):
#     z = 10
    print('file z::: %s' % str(z))
    print(targetfiles[z])

    dfs = tabula.read_pdf(targetfiles[z], multiple_tables=True)    
    for df in dfs[:2]:
        country_name = df.iloc[0, 0]
        ndf = df.dropna(how='all', axis='columns')
        nc = ndf[2:4].apply(lambda x: ' '.join(map(str, x.tolist())).replace('nan', '').strip()).tolist()
        
        tc.append(nc)
        odf.append(df)
               
        if nc.__contains__('#'):
            nc[0] = '# Name'
            nc[1] = ''
                         
        tdf = ndf[4:29]
        tdf.columns = nc
               
        if not nc.__contains__('H') and [x for x in nc if x.__contains__('H ')].__len__() > 0:
            hcol = [x for x in nc if x.__contains__('H ')][0]
            tdf[hcol] = tdf[hcol].apply(lambda x: str(x).split(' ')[0] if not str(x).__contains__('(') else ' '.join(str(x).split(' ')[0:2]))
            nc[nc.index(hcol)] = 'H'
            tdf.columns = nc        

        start_idx_col = 'DOB'  

        if nc.__contains__('Pos DOB'):
            tdf = tdf.reset_index(drop=True).join(pd.DataFrame(tdf['Pos DOB'].apply(lambda x: str(x).split(' ')).tolist(), columns = ['Pos', 'DOB']))
            start_idx_col = 'Pos DOB'

        if not nc.__contains__('Club'):
            nc = tdf.columns.tolist()
            tdf['Club H'] = tdf.iloc[:, nc.index(start_idx_col)+1:nc.index('Club H')+1].apply(lambda x: ' '.join(map(str, x.tolist())).replace('nan', '').strip(), axis=1)
            tdf = tdf.reset_index(drop=True).join(pd.DataFrame(tdf['Club H'].apply(lambda x: str(x).split(') ')).tolist(), columns = ['Club', 'H']))
            tdf.Club = tdf.Club.apply(lambda x: x + ')' if x != '' else x)
        else:
            nc = tdf.columns.tolist()
            tdf.Club = tdf.iloc[:, nc.index(start_idx_col)+1:nc.index('Club')+1].apply(lambda x: ' '.join(map(str, x.tolist())).replace('nan', '').strip(), axis=1)            

        tdf = tdf.reset_index(drop=True).join(pd.DataFrame(tdf['# Name'].apply(lambda x: str(x).split(' ', 1)).tolist(), columns = ['player_number', 'player_name']))

        tdf.Club = tdf.apply(lambda x: (x.Club + str(x.H).split(' ')[0]) if str(x.H).__contains__('(') else x.Club, axis=1)
        tdf.H = tdf.apply(lambda x: str(x.H).split(' ')[1] if str(x.H).__contains__('(') else x.H, axis=1)

        ldf[country_name] = tdf[['player_number', 'player_name', 'Pos', 'DOB', 'Club', 'H']]

        tdf[['player_number', 'player_name', 'Pos', 'DOB', 'Club', 'H']]
    
#     break

In [8]:
ldf.keys().__len__(), ldf.keys()

(32,
 odict_keys(['Russia (RUS)', 'Saudi Arabia (KSA)', 'Egypt (EGY)', 'Uruguay (URU)', 'Portugal (POR)', 'Spain (ESP)', 'Morocco (MAR)', 'IR Iran (IRN)', 'France (FRA)', 'Australia (AUS)', 'Peru (PER)', 'Denmark (DEN)', 'Argentina (ARG)', 'Iceland (ISL)', 'Croatia (CRO)', 'Nigeria (NGA)', 'Brazil (BRA)', 'Switzerland (SUI)', 'Costa Rica (CRC)', 'Serbia (SRB)', 'Germany (GER)', 'Mexico (MEX)', 'Sweden (SWE)', 'Korea Republic (KOR)', 'Belgium (BEL)', 'Panama (PAN)', 'Tunisia (TUN)', 'England (ENG)', 'Poland (POL)', 'Senegal (SEN)', 'Colombia (COL)', 'Japan (JPN)']))

In [ ]:
for i, k in enumerate(ldf.keys()):
    print(k)
    display(ldf[k])

In [13]:
ldf['Russia (RUS)'].columns

Index(['player_number', 'player_name', 'Pos', 'DOB', 'Club', 'H'], dtype='object')

In [34]:
def insert_player_info(player_info):
#     print(player_info)
    name = player_info['player_name']
    team_id = player_info['team_id']
    num = player_info['player_num']
    position = player_info['player_position']
    club = player_info['player_club']
    birth = player_info['player_birth']
    height = player_info['player_height']
    
    curs = CONN.cursor()
    find_sql = """select id from player where num=%s and team_id=%s"""
    curs.execute(find_sql, (num, team_id))
    player_id = curs.fetchone()
#     print(player_id)
    if player_id is not None:
        update_sql = """update player set club=%s, birth=%s, height=%s, position=%s where id=%s"""
        curs.execute(update_sql, (club, birth, height, position, player_id))
        CONN.commit()
    else:
        insert_sql = """insert into player(team_id, num, name, club, birth, height, position) values(%s,%s,%s,%s,%s,%s,%s)"""
        curs.execute(insert_sql, (team_id, num, name, club, birth, height, position))
        CONN.commit()
        print('INSERTTED+++++++++++++++++++++++++++++++')
        print(player_info)

In [ ]:
for key in ldf.keys():
    country_id = countries[key.split('(')[0].strip().lower()]
    print(country_id)
    team_df = ldf[key].iloc[[i for i in range(0, 25) if i != 11]]
#     display(team_df)
    
    for ind, row in team_df.iterrows():
        if ind==24:
            name = row['player_name'].split('(')[0].strip()
#             position = row['player_number'] #coach
#             insert_player_info({'team_id':country_id, 'name':name, 'position':position})
        else:
            name = row['player_name'].strip()
            player_num = row['player_number'].strip()
            position = row['Pos'].strip()
            birth = row['DOB'].strip()
            club = row['Club']
            height = row['H']
            save_params = {'player_num':player_num,
                           'team_id':country_id,
                          'player_birth':birth.strip(),
                          'player_club':club.strip(),
                           'player_position':position.strip(),
                           'player_name':name,
                          'player_height':height}
#             print(save_params)
            insert_player_info(save_params)


In [40]:
regex = re.search('([0-9A-z]*) ([A-Z]*)', 'Daegu FC (KOR)')
regex.group(0)

'Daegu FC'

In [36]:
ldf['Egypt (EGY)'].iloc[[i for i in range(0, 25) if i != 11]]
# ldf['Russia (RUS)'].iloc[[i for i in range(0, 25) if i != 11]]

,player_number,player_name,Pos,DOB,Club,H
0,23,MOHAMED ELSHENAWY,GK,18/12/88,Al Ahly Cairo (EGY),191
1,2,ALI GABR,DF,01/01/89,West Bromwich Albion.(ENG),191
2,6,AHMED HEGAZY,DF,25/01/91,West Bromwich Albion.(ENG),194
3,7,AHMED FATHI,DF,10/11/84,Al Ahly Cairo (EGY),176
4,8,TAREK HAMED,MF,24/10/88,Zamalek (EGY),169
5,9,MARWAN MOHSEN,FW,26/02/89,Al Ahly Cairo (EGY),185
6,13,MOHAMED ABDELSHAFY,DF,01/07/85,Al Fath (KSA),171
7,17,MOHAMED ELNENY,MF,11/07/92,Arsenal (ENG),181
8,19,ABDALLA SAID,MF,13/07/85,Kuopion Palloseura (FIN),176
9,21,TREZEGUET,MF,01/10/94,Kasimpasa SK (TUR),181
